In [101]:
import sqlite3

In [102]:
conn = sqlite3.connect('playlist.db')
cur = conn.cursor()

In [70]:
cur.execute('''
        DROP TABLE IF EXISTS ARTIST
        CREATE TABLE ARTIST(
            PK INTEGER PRIMARY KEY,
            NAME TEXT NOT NULL DEFAULT '가수'
        );
''')
#PIRMARY KEY = UNIQUE NOT NULL AUTO INCREMENT

OperationalError: near "CREATE": syntax error

In [ ]:
cur.executescript('''
                DROP TABLE IF EXISTS ALBUM;
                CREATE TABLE ALBUM(
                  PK INTEGER PRIMARY KEY,
                  NAME TEXT,
                  FK INTEGER NOT NULL
                );
                
                DROP TABLE IF EXISTS GENRE;
                CREATE TABLE GENRE(
                  PK INTEGER PRIMARY KEY,
                  NAME TEXT NOT NULL
                );
                  
                DROP TABLE IF EXISTS TRACK;
                CREATE TABLE TRACK(
                  PK INTEGER PRIMARY KEY,
                  NAME TEXT NOT NULL,
                  LENGTH INTEGER DEFAULT 0,
                  RATING INTEGER DEFAULT 0,
                  COUNT INTEGER DEFAULT 0,
                  AFK INTEGER NOT NULL,
                  GFK INTEGER NOT NULL
                );
''')

In [ ]:
cur.executemany('''
            INSERT INTO ARTIST VALUES(NULL, ?)
''',[['수지'], ['박효신'], ['아이유']])

#execute 하나만 하면 여러개의 insert문을 사용할 수 없다
#또한, Qmark를 쓸 때도 list안에 있는 값들은 iterable하게 감싸줘야 하므로 
#[a, b, c] 에서 [[a], [b], [c]]로 해줘야 함

In [ ]:
conn.commit()

In [ ]:
cur.executemany('''
            INSERT INTO GENRE VALUES(NULL, ?)
''',[['발라드'], ['R&B'], ['락']])
conn.commit()

In [ ]:
#artist 찾아서 pk 불러오고 ablum pk를 fk로 바꿔서 insert 하는 함수
def addAlbum(artist, album):
    cur.execute('SELECT PK FROM ARTIST WHERE NAME LIKE ?', ['%'+artist+'%']) #중간에 artist와 일치한, 포함한 모든 결과값
    PK = cur.fetchone()

    if PK:
        FK = PK[0]
        cur.execute('INSERT INTO ALBUM VALUES(NULL, ?, ?)', (album, FK))
    return PK, artist, album, cur.lastrowid

addAlbum('아이유', '아이유앨범')

((3,), '아이유', '아이유앨범', 3)

In [ ]:
#album - PK1, genre => pk2
#INSERT TRACK, AFK=PK1, GFK = PK2
def addTrack(album, genre, track, length=0, rating=0, count=0):
    cur.execute('SELECT PK FROM ALBUM WHERE NAME LIKE ?', ['%'+album+'%']) #중간에 artist와 일치한, 포함한 모든 결과값
    PK = cur.fetchone()

    cur.execute('SELECT PK FROM GENRE WHERE NAME LIKE ?', ['%'+genre+'%'])
    PK2 = cur.fetchone()

    if PK != None and PK2 != None:
        AFK = PK[0]
        GFK = PK2[0]
        cur.execute('''
                    INSERT INTO TRACK VALUES(NULL, :name, :length, :rating, :count, :afk, :gfk)
                    ''', {'name': track, 'length': length, 'rating': rating,'count': count, 'afk': AFK, 'gfk':GFK})
        conn.commit()
    return AFK, GFK

addTrack('박효신', 'R', '박효신노래3')

(2, 2)

In [ ]:
cur.execute('''
            UPDATE TRACK
            SET AFK=?
            WHERE NAME=?
''', (3, '아이유노래2'))
conn.commit()

In [ ]:
cur.execute('SELECT * FROM TRACK')
cur.fetchall()

[(1, '아이유노래3', 0, 0, 0, 3, 1),
 (2, '아이유노래2', 0, 0, 0, 3, 2),
 (3, '아이유노래3', 0, 0, 0, 3, 2),
 (4, '박효신노래1', 0, 0, 0, 2, 2),
 (5, '박효신노래2', 0, 0, 0, 2, 2),
 (6, '박효신노래3', 0, 0, 0, 2, 2)]

In [ ]:
#아티스트와 앨범 섞기
cur.execute('SELECT DISTINCT(NAME) FROM ARTIST') #중복되는 사람 구별하기 위해서 DINSTINCT 사용
cur.fetchall()

cur.execute('SELECT * FROM ALBUM')
cur.fetchall()

cur.execute('''
            SELECT ARTIST.NAME, ALBUM.NAME
            FROM ARTIST
            LEFT JOIN ALBUM
            ON ALBUM.FK = ARTIST.PK
''')
cur.fetchall()

#SELECT _____                       1ST
#FROM ARTIST                        2ND
#INNER JOIN 누구                    3RD
#ON ALBUM.FK = ARTIST.PK            4TH
#SELECT ARTIST.NAME, ALBUM.NAME     5TH

[('수지', '수지앨범'), ('박효신', '박효신앨범'), ('아이유', '아이유노래1'), ('아이유', '아이유앨범')]

In [105]:
cur.execute('''
            SELECT ARTIST.NAME, COUNT(ALBUM.NAME)
            FROM ARTIST
            LEFT JOIN ALBUM
            ON ALBUM.FK = ARTIST.PK
            GROUP BY ARTIST.NAME    
            ORDER BY ARTIST.NAME ASC
''')
cur.fetchall()
#GROUP BY로 안묶어주면 결과가 이상해짐 즉, 집계함수를 쓸 경우에는 어떤 기준으로 묶을지 GROUP BY로 묶어줘야함

[('수지', 4)]

In [ ]:
#ALBUM - TRACK
cur.execute('''
            SELECT ALBUM.NAME, COUNT(TRACK.NAME)
            FROM ALBUM
            LEFT JOIN TRACK
            ON TRACK.AFK=ALBUM.PK
            GROUP BY ALBUM.PK
''')
cur.fetchall()

[('수지앨범', 0), ('박효신앨범', 3), ('아이유앨범', 3), ('아이유노래1', 0)]

In [ ]:
cur.execute('''
            SELECT GENRE.NAME, COUNT(TRACK.NAME)
            FROM GENRE
            LEFT JOIN TRACK
            ON TRACK.GFK=GENRE.PK
            GROUP BY GENRE.PK
''')
cur.fetchall()

[('발라드', 1), ('R&B', 5), ('락', 0)]

In [ ]:
#ARTIST - ALBUM - TRACK
cur.execute('''
            SELECT A.NAME, B.NAME, C.NAME
            FROM ARTIST AS A
            LEFT JOIN ALBUM AS B
            ON B.FK = A.PK
            LEFT JOIN TRACK AS C
            ON C.AFK=B.PK
''')
cur.fetchall()

cur.execute('''
            SELECT A.NAME, B.NAME, B.TRNAME
            FROM ARTIST AS A
            LEFT JOIN (
                SELECT ALBUM.FK, ALBUM.NAME AS NAME, TRACK.NAME AS TRNAME
                FROM ALBUM
                LEFT JOIN TRACK
                ON TRACK.AFK = ALBUM.PK
            ) AS B
            ON B.FK = A.PK
''')
cur.fetchall()

cur.execute('''
            SELECT A.NAME, COUNT(C.NAME)
            FROM ARTIST AS A, ALBUM AS B, TRACK AS C
            WHERE A.PK=B.FK AND B.PK=C.AFK
            GROUP BY A.NAME
''')
cur.fetchall()

[('아이유', '아이유앨범', '아이유노래3'),
 ('아이유', '아이유앨범', '아이유노래2'),
 ('아이유', '아이유앨범', '아이유노래3'),
 ('박효신', '박효신앨범', '박효신노래1'),
 ('박효신', '박효신앨범', '박효신노래2'),
 ('박효신', '박효신앨범', '박효신노래3')]

In [ ]:
#가수-앨범-장르-노래
cur.execute('''
            SELECT A.NAME, B.NAME, C.NAME, D.NAME
            FROM ARTIST AS A, ALBUM AS B, GENRE AS C, TRACK AS D
            WHERE A.PK=B.FK AND B.PK=D.AFK AND C.PK=D.GFK
            
''')
cur.fetchall()

[('아이유', '아이유앨범', '발라드', '아이유노래3'),
 ('아이유', '아이유앨범', 'R&B', '아이유노래2'),
 ('아이유', '아이유앨범', 'R&B', '아이유노래3'),
 ('박효신', '박효신앨범', 'R&B', '박효신노래1'),
 ('박효신', '박효신앨범', 'R&B', '박효신노래2'),
 ('박효신', '박효신앨범', 'R&B', '박효신노래3')]

In [ ]:
cur.execute('''
            SELECT A.NAME, B.NAME, C.NAME, COUNT(D.NAME)
            FROM ARTIST AS A, ALBUM AS B, GENRE AS C, TRACK AS D
            WHERE A.PK=B.FK AND B.PK=D.AFK AND C.PK=D.GFK
            GROUP BY A.PK, B.PK, C.PK                           #A.PK로 묶고, 그 다음에 B.PK로 다시 묶고 (묶이는게 존재하면), C.PK로 다시 묶고 (존재하면)
''')
cur.fetchall()

[('박효신', '박효신앨범', 'R&B', 3),
 ('아이유', '아이유앨범', '발라드', 1),
 ('아이유', '아이유앨범', 'R&B', 2)]

In [ ]:
#VIEW 쓰는 방법
cur.execute('''
    CREATE VIEW ALBUM_TRACK AS
            SELECT ALBUM.FK, ALBUM.NAME AS NAME, TRACK.NAME AS TNAME
            FROM ALBUM
            LEFT JOIN TRACK
            ON TRACK.AFK = ALBUM.PK
''')

cur.execute('''
    SELECT A.NAME, B.NAME, B.TNAME
            FROM ARTIST AS A
            LEFT JOIN ALBUM_TRACK AS B
            ON B.FK = A.PK
''')

cur.fetchall()

[('수지', '수지앨범', None),
 ('박효신', '박효신앨범', '박효신노래1'),
 ('박효신', '박효신앨범', '박효신노래2'),
 ('박효신', '박효신앨범', '박효신노래3'),
 ('아이유', '아이유노래1', None),
 ('아이유', '아이유앨범', '아이유노래2'),
 ('아이유', '아이유앨범', '아이유노래3'),
 ('아이유', '아이유앨범', '아이유노래3')]

In [ ]:
conn.close()

In [ ]:
#ORM
#SNS - 게시글(해쉬태그) 등록/수정/삭제

conn = sqlite3.connect('sns.db')
cur = conn.cursor()

In [89]:
cur.executescript('''
    CREATE TABLE POST(
        PK INTEGER PRIMARY KEY
    );
                  
    CREATE TABLE HASHTAG(
        PK INTEGER PRIMARY KEY,
        TAG TEXT NOT NULL,
        COUNT INTEGER DEFAULT 0
    );
                  
    CREATE TABLE POSTTAG(
        PFK INTEGER NOT NULL,
        TFK INTEGER NOT NULL
    );
''')

In [91]:
cur.execute('''
    ALTER TABLE POST
    ADD COLUMN CONTENT TEXT
''')

In [ ]:
conn.close()

In [ ]:
conn = sqlite3.connect('playlist.db')
cur = conn.cursor()

In [ ]:
cur.execute('''
            ALTER TABLE ARTIST
            ADD COLUMN temp TEXT NOT NULL DEFAULT ''
''')

In [ ]:
conn.close()

In [87]:
conn = sqlite3.connect('sns.db')
cur = conn.cursor()

In [88]:
cur.execute('DROP TABLE POST')
cur.execute('DROP TABLE HASHTAG')
cur.execute('DROP TABLE POSTTAG')
conn.commit()

In [97]:
#post 등록 프로세스 생성
def newPosting(content, *tags):
    #1
    cur.execute('INSERT INTO POST VALUES(NULL, ?)', (content,))
    conn.commit()

    cur.execute('SELECT PK FROM POST ORDER BY PK DESC LIMIT 0, 1')
    fk = cur.fetchone()[0]

    #2
    fkList = list()
    for tag in tags:
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        if cur.fetchone() is None: #태그풀에 있는지 검사 없으면 실행
            cur.execute('INSERT INTO HASHTAG (TAG) VALUES (?)', (tag,))
            conn.commit()
        #무조건 태그풀 존재 왜냐면 없으면 넣었으니깐
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        PK = cur.fetchone()[0]
        fkList.append(PK)
        cur.execute('UPDATE HASHTAG SET COUNT=COUNT+1 WHERE PK=?', (PK,))
        conn.commit()

    #3
    for FK in fkList:
        cur.execute('INSERT INTO POSTTAG VALUES(?, ?)', (fk, FK))

    # conn.commit()
    return fk, fkList

newPosting('내용1', 'python', 'alchemy')

(6, [1, 2])

In [ ]:
# 등록 - 1 (내용 추가)
cur.execute('INSERT INTO POST VALUES(NULL, ?)', (content,))
# cur.lastrowid => 얘는 신뢰가 안됨 pk가 아니므로
conn.commit()
# cur.execute('SELECT PK FROM POST WHERE CONTENT=?', (content,))
# or (좀 그렇긴 함 왜냐하면 비교할 애들이 많아서)
cur.execute('SELECT PK FROM POST OREDER BY PK DESC LIMIT 0, 1')
PK = cur.fetchone()[0]

In [ ]:
#등록 - 2 태그 추가
fkList = list()
for tag in tags:
    cur.execute('SELECT PK FROM HASHTAG TAG=?', (tag,))
    PK = cur.fetchone()
    if PK:
        PK = PK[0]
        fkList.append(PK)
        cur.execute('UPDATE HASHTAG SET COUNT=COUNT+1 WHERE PK=?', (PK,))
        conn.commit()
    else:
        cur.execute('INSERT INTO HASHTAG VALUES (NULL, ?, 1)', (tag,))
        conn.commit()

In [ ]:
#등록 - 3 (1,2 합쳐서)
PK, fkList
for FK in fkList:
    cur.executemany('''
                    INSERT INTO POSTTAG
                    VALUES(?,?)
    ''', (PK, FK))

#위에 함수로

In [ ]:
for tag in tags:
    cur.execute('SELECT PK FROM HASHTAG TAG=?', (tag,))
    if cur.fetchone() is None: #태그풀에 있는지 검사 없으면 실행
        cur.execute('INSERT INTO HASHTAG VALUES (NULL, ?, 0)', (tag,))
        conn.commit()
    #무조건 태그풀 존재 왜냐면 없으면 넣었으니깐
    cur.execute('SELECT PK FROM HASHTAG TAG=?', (tag,))
    PK = cur.fetchone()[0]
    fkList.append(PK)
    cur.execute('UPDATE HASHTAG SET COUNT=COUNT+1 WHERE PK=?', (PK,))
    conn.commit()

In [98]:
#수정
def modifyPost(content, *tags):
    cur.execute('SELECT PK FROM POST WHERE CONTENT=?', (content,))
    PPK = cur.fetchone()[0]
    #1. 게시글 찾음

    #2. 게시글에 달린 해시태그 목록
    cur.execute('SELECT TFK FROM POSTTAG WHERE PFK=?',(PPK,))
    tagList = list()
    for row in cur.fetchall():
        tagList.append(row[0])

    #3. 해시태그 찾는 과정
    tpkList = list()
    for tag in tags:
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        if cur.fetchone() is None:
            cur.execute('INSERT INTO HASHTAG (TAG) VALUES (?)', (tag,))
            conn.commit()
        
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        TPK = cur.fetchone()[0]
        tpkList.append(TPK)

        #기존 태그 일치하는지 비교
        if TPK not in tagList:
            cur.execute('UPDATE HASHTAG SET COUNT=COUNT+1 WHERE PK=?', (TPK,))
            cur.execute('INSERT INTO POSTTAG VALUES(?,?)', (PPK, TPK))
            conn.commit()

    #4. 삭제된 해시태그 찾는과정
    for tag in tagList:
        if tag not in tpkList:
            cur.execute('UPDATE HASHTAG SET COUNT = COUNT - 1 WHERE PK=?', (tag,))
            cur.execute('DELETE FROM POSTTAGE WHERE PFK=? AND TFK=?', (PPK, tag))
            conn.commit()

modifyPost('내용1', 'python', 'orm')

# RDBMS(SQLite) - not a database

Entity and Relation - Entity is a row and Relation is the table

Table1         Table2
PK     ------> FK

Minimize duplicate entities for each tables

Using SQL order RDBMS(SQLite) to post, select, etc.

### Order to write SQL

1. SELECT _____                       
2. FROM ARTIST                        
3. INNER JOIN TABLE                    
4. ON ALBUM.FK = ARTIST.PK            
5. SELECT ARTIST.NAME, ALBUM.NAME     

### Some SQL

For INSERT query, the numbers of columns and values must be same

LIKE - %(wildcard - anything)

GROUP BY / ORDER BY / JOIN (CROSS (all possible sets), INNER (A n B), LEFT (A), RIGHT(B), OUTER) ON, AS (select one table and name such as A)

VIEW - SQL command that often using

```
cur.execute('''
    CREATE VIEW ALBUM_TRACK AS
            SELECT ALBUM.FK, ALBUM.NAME AS NAME, TRACK.NAME AS TNAME
            FROM ALBUM
            LEFT JOIN TRACK
            ON TRACK.AFK = ALBUM.PK
''')
```

### UNIQUE and DISTINCT

Unique is used when making a column

Distinct is a function that dividing data by unique values

### Join
join is faster than duplicate sql query statement but the code will be longer, hard to change

So we use view table

If we do not put GROUP BY and call COUNT(ALBUM.NAME) then, the result will show only one value

* executescript - execute bunch of SQL statement
* executemany - execute SQL statement with iterable indicator
* execute - execute only one SQL statement

If we want to search like using %

```
cur.execute('SELECT PK FROM ARTIST WHERE NAME LIKE ?', ['%'+artist+'%']) 
```

Search the name value that contains artist value

Ways to join tables

ex) ARTIST - ALBUM - TRACK

1. 
```
cur.execute('''
            SELECT A.NAME, B.NAME, C.NAME
            FROM ARTIST AS A
            LEFT JOIN ALBUM AS B
            ON B.FK = A.PK
            LEFT JOIN TRACK AS C
            ON C.AFK=B.PK
''')
```

2. 
```
cur.execute('''
            SELECT A.NAME, B.NAME, B.TRNAME
            FROM ARTIST AS A
            LEFT JOIN (
                SELECT ALBUM.FK, ALBUM.NAME AS NAME, TRACK.NAME AS TRNAME
                FROM ALBUM
                LEFT JOIN TRACK
                ON TRACK.AFK = ALBUM.PK
            ) AS B
            ON B.FK = A.PK
''')
```

3. 
```
cur.execute('''
            SELECT A.NAME, COUNT(C.NAME)
            FROM ARTIST AS A, ALBUM AS B, TRACK AS C
            WHERE A.PK=B.FK AND B.PK=C.AFK
            GROUP BY A.NAME
''')
```

The 3rd way is much slower so we have to avoid it

### How to make SNS post func

```
conn = sqlite3.connect('sns.db')
cur = conn.cursor()
#Connect and load cursor

cur.executescript('''
    CREATE TABLE POST(
        PK INTEGER PRIMARY KEY
    );
                  
    CREATE TABLE HASHTAG(
        PK INTEGER PRIMARY KEY,
        TAG TEXT NOT NULL,
        COUNT INTEGER DEFAULT 0
    );
                  
    CREATE TABLE POSTTAG(
        PFK INTEGER NOT NULL,
        TFK INTEGER NOT NULL
    );
''')

#Create Tables

cur.execute('''
    ALTER TABLE POST
    ADD COLUMN CONTENT TEXT
''')

#Post a new posting
def newPosting(content, *tags):
    #1
    cur.execute('INSERT INTO POST VALUES(NULL, ?)', (content,))
    conn.commit()

    cur.execute('SELECT PK FROM POST ORDER BY PK DESC LIMIT 0, 1')
    fk = cur.fetchone()[0]

    #2
    fkList = list()
    for tag in tags:
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        if cur.fetchone() is None: #Determine whether tag exists
            cur.execute('INSERT INTO HASHTAG (TAG) VALUES (?)', (tag,))
            conn.commit()
        #Tag must be exists since if there is no tag math, we already put them in
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        PK = cur.fetchone()[0]
        fkList.append(PK)
        cur.execute('UPDATE HASHTAG SET COUNT=COUNT+1 WHERE PK=?', (PK,))
        conn.commit()

    #3
    for FK in fkList:
        cur.execute('INSERT INTO POSTTAG VALUES(?, ?)', (fk, FK))

    # conn.commit()
    return fk, fkList

newPosting('내용1', 'python', 'alchemy')

#Modify Post
def modifyPost(content, *tags):
    cur.execute('SELECT PK FROM POST WHERE CONTENT=?', (content,))
    PPK = cur.fetchone()[0]
    #1. Find posting

    #2. Hashtag that is on the posting
    cur.execute('SELECT TFK FROM POSTTAG WHERE PFK=?',(PPK,))
    tagList = list()
    for row in cur.fetchall():
        tagList.append(row[0])

    #3. Finding hashtag
    tpkList = list()
    for tag in tags:
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        if cur.fetchone() is None:
            cur.execute('INSERT INTO HASHTAG (TAG) VALUES (?)', (tag,))
            conn.commit()
        
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?', (tag,))
        TPK = cur.fetchone()[0]
        tpkList.append(TPK)

        #Compare whether tag already exists is same or not
        if TPK not in tagList:
            cur.execute('UPDATE HASHTAG SET COUNT=COUNT+1 WHERE PK=?', (TPK,))
            cur.execute('INSERT INTO POSTTAG VALUES(?,?)', (PPK, TPK))
            conn.commit()

    #4. Finding deleted hashtag
    for tag in tagList:
        if tag not in tpkList:
            cur.execute('UPDATE HASHTAG SET COUNT = COUNT - 1 WHERE PK=?', (tag,))
            cur.execute('DELETE FROM POSTTAGE WHERE PFK=? AND TFK=?', (PPK, tag))
            conn.commit()

modifyPost('내용1', 'python', 'orm')
```